<a href="https://colab.research.google.com/github/martinpdab/tugasscdl/blob/main/Skenario1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sklearn

In [2]:
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [3]:
#memakai dataset
URL = 'https://raw.githubusercontent.com/martinpdab/tugasscdl/main/data%20fix.csv' 
dataframe = pd.read_csv(URL)
dataframe

,Period,TE,HU,RF,WS,PD,ABJ,LAG0,LAG1,LAG2,LAG3,LAG4,LAG5,LAG6,LAG7,LAG8,LAG9,LAG10,LAG11,LAG12,OS
0,Jan-10,26.02,84.74,10.52,0.84,792.86,86.96,1,4,6,4,4,1,4,3,3,5,2,1,4,0
1,Feb-10,26.32,84.64,12.04,0.86,793.45,86.72,3,1,4,6,4,4,1,4,3,3,5,2,1,1
2,Mar-10,26.39,85.00,8.65,0.55,794.04,86.50,0,3,1,4,6,4,4,1,4,3,3,5,2,0
3,Apr-10,26.29,86.17,10.37,0.30,794.63,86.28,1,0,3,1,4,6,4,4,1,4,3,3,5,0
4,May-10,26.57,85.61,7.35,0.58,795.22,86.08,0,1,0,3,1,4,6,4,4,1,4,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Aug-18,24.27,76.87,0.00,1.39,846.18,96.29,1,4,3,3,5,2,1,4,1,1,1,2,3,0
104,Sep-18,25.50,75.73,0.59,1.03,846.59,93.34,4,1,4,3,3,5,2,1,4,1,1,1,2,1
105,Oct-18,26.58,74.94,0.04,1.39,847.01,96.10,4,4,1,4,3,3,5,2,1,4,1,1,1,1
106,Nov-18,26.49,82.28,10.53,1.00,847.42,96.38,6,4,4,1,4,3,3,5,2,1,4,1,1,1


In [4]:
# mengambil kolom TE, HU, RF, WS, PD, ABJ, OS dan mengacak dataset
data = dataframe[['TE', 'HU', 'RF', 'WS', 'PD', 'ABJ', 'OS']]
data = shuffle(data)
data.head()

,TE,HU,RF,WS,PD,ABJ,OS
38,26.12,84.72,9.80,0.73,815.16,100.000000,1
32,25.15,76.70,0.00,1.73,811.83,97.060000,0
70,27.81,77.67,0.34,0.60,831.78,84.333333,0
107,26.14,85.45,10.41,0.87,847.84,95.350000,1
63,26.09,86.60,6.24,0.10,828.07,89.484880,0


In [5]:
#train-test split 80-20
train, test = train_test_split(data, test_size=0.2)
print(len(train), 'banyak data train')
print(len(test), 'banyak data test')

#train-test split 70-30
train1, test1 = train_test_split(data, test_size=0.3)
print(len(train1), 'banyak data train')
print(len(test1), 'banyak data test')

86 banyak data train
22 banyak data test
75 banyak data train
33 banyak data test


In [6]:
#misahin label
def df_to_dataset(data, batch_size=32):
  data = data.copy()
  labels = data.pop('OS')
  ds = tf.data.Dataset.from_tensor_slices((dict(data), labels))
  ds = ds.batch(batch_size)
  return ds

In [17]:
batch_size = 64 # A small batch sized is used for demonstration purposes
#80-20 split
train_ds = df_to_dataset(train, batch_size=batch_size)
test_ds = df_to_dataset(test, batch_size=batch_size)

#70-30 split
train_ds1 = df_to_dataset(train1, batch_size=batch_size)
test_ds1 = df_to_dataset(test1, batch_size=batch_size)

In [14]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('TE:', feature_batch['TE'])
  print('HU:', feature_batch['HU'])
  print('OS:', label_batch)

Every feature: ['TE', 'HU', 'RF', 'WS', 'PD', 'ABJ']
TE: tf.Tensor([26.76 25.5  24.44 24.27 25.16], shape=(5,), dtype=float64)
HU: tf.Tensor([81.06 75.73 80.39 76.87 81.63], shape=(5,), dtype=float64)
OS: tf.Tensor([0 1 0 0 0], shape=(5,), dtype=int64)


In [15]:
feature_columns = []

for header in ['TE', 'HU', 'RF', 'WS', 'PD', 'ABJ']:
  feature_columns.append(feature_column.numeric_column(header))

In [16]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer

In [18]:
#train 80-20
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(32, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=test_ds,
          epochs=75)

Epoch 1/75
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1/2 [==============>...............] - ETA: 0s - loss: 161.4616 - accuracy: 0.2500WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'TE': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=float64>, 'HU': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=float64>, 'RF': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=float64>, 'WS': <tf.Tensor 'ExpandDims_5:0' shape=(None, 1) dtype=float64>, 'PD': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=float64>, 'ABJ': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=float64>}
Consider rewriting this model with the Functional API.
2/2 [==============================] - 1s 162ms/step - loss: 149.7903 - accuracy: 0.2791 - val_loss: 113.8807 - val_accuracy: 0.2273
Epoch 2/75
2/2 [==============================] - 0s 17ms/step - loss: 100.7860 - accu

In [19]:
#train 70-30
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(32, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds1,
          validation_data=test_ds1,
          epochs=75)

Epoch 1/75
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1/2 [==============>...............] - ETA: 0s - loss: 15.1696 - accuracy: 0.7188WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'TE': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=float64>, 'HU': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=float64>, 'RF': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=float64>, 'WS': <tf.Tensor 'ExpandDims_5:0' shape=(None, 1) dtype=float64>, 'PD': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=float64>, 'ABJ': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=float64>}
Consider rewriting this model with the Functional API.
2/2 [==============================] - 1s 307ms/step - loss: 14.4006 - accuracy: 0.7067 - val_loss: 1.4469 - val_accuracy: 0.7879
Epoch 2/75
2/2 [==============================] - 0s 17ms/step - loss: 2.8943 - accuracy: 